In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_fwf('./wili dataset/x_train.txt', header=None)
X_train = df[[0]]
df = pd.read_fwf('./wili dataset/x_test.txt', header=None)
X_test = df[[0]]

In [16]:
target = pd.read_fwf('./wili dataset/y_train.txt',header = None)
y_train = target[0]
target = pd.read_fwf('./wili dataset/y_test.txt',header = None)
y_test = target[0]

In [18]:
X_train.head()

,0
0,Klement Gottwaldi surnukeha palsameeriti ning ...
1,"Sebes, Joseph; Pereira Thomas (1961) (pÃ¥ eng)..."
2,à¤­à¤¾à¤°à¤¤à¥€à¤¯ à¤¸à¥�à¤µà¤¾à¤¤à¤¨à¥�à¤¤à¥�...
3,AprÃ¨s lo cort periÃ²de d'establiment a BasilÃ...
4,à¸–à¸™à¸™à¹€à¸ˆà¸£à¸´à¸�à¸�à¸£à¸¸à¸‡ (à¸­à¸±à¸...


In [19]:
X_test.head()

,0
0,Ne l fin de l seclo XIX l Japon era inda Ã§con...
1,Schiedam is gelegen tussen Rotterdam en Vlaard...
2,"Ð“IÑƒÑ€ÑƒÑ�Ð°Ð· Ð±Ð°Ñ‚Ð°Ð»ÑŒÐ¾Ð½Ð°Ð», Ð³ÑŒÐ¾Ñ€..."
3,à²°à²¾à²œà³�à²¯à²¶à²¾à²¸à³�à²¤à³�à²°à²¦ à²ªà²¿...
4,Halukum adalah kelenjar tiroid nang menonjol d...
